In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot

%matplotlib inline

In [2]:
auctions = pd.read_csv('auctions.csv.gzip', compression='gzip')

# Hecho un vistazo de los datos para depurar

### Auctions:

In [3]:
#Veo un poco de informacion sobre el DataFrame

auctions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19571319 entries, 0 to 19571318
Data columns (total 7 columns):
auction_type_id    float64
country            int64
date               object
device_id          int64
platform           int64
ref_type_id        int64
source_id          int64
dtypes: float64(1), int64(5), object(1)
memory usage: 1.0+ GB


In [4]:
##### Se puede ver que este Dataframe utiliza mucha memoria

In [5]:
auctions.head()

,auction_type_id,country,date,device_id,platform,ref_type_id,source_id
0,NaN,6333597102633388268,2019-03-11 14:18:33.290763,6059599345986491085,1,1,0
1,NaN,6333597102633388268,2019-03-11 14:18:34.440157,1115994996230693426,1,1,0
2,NaN,6333597102633388268,2019-03-11 14:18:35.862360,7463856250762200923,1,1,0
3,NaN,6333597102633388268,2019-03-11 14:18:36.167163,7829815350267792040,1,1,0
4,NaN,6333597102633388268,2019-03-11 14:18:37.728590,1448534231953777480,1,1,0


In [6]:
auctions['date'] = pd.to_datetime(auctions['date'])

In [7]:
auctions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19571319 entries, 0 to 19571318
Data columns (total 7 columns):
auction_type_id    float64
country            int64
date               datetime64[ns]
device_id          int64
platform           int64
ref_type_id        int64
source_id          int64
dtypes: datetime64[ns](1), float64(1), int64(5)
memory usage: 1.0 GB


In [9]:
auctions.count()

auction_type_id           0
country            19571319
date               19571319
device_id          19571319
platform           19571319
ref_type_id        19571319
source_id          19571319
dtype: int64

In [10]:
auctions['auction_type_id'].isnull().sum()

19571319

### Todas las filas son NaN, voy a descartar esta columna (auction_type_id) para achicar el tamaño

In [11]:
auctions = auctions.drop(columns='auction_type_id')

In [12]:
auctions['platform'].value_counts()

1    15541825
2     4029494
Name: platform, dtype: int64

### Por la gran cantidad de valores con plataforma 1, asumo que corresponde a Android y 2 a iOS.

In [13]:
auctions.head()

,country,date,device_id,platform,ref_type_id,source_id
0,6333597102633388268,2019-03-11 14:18:33.290763,6059599345986491085,1,1,0
1,6333597102633388268,2019-03-11 14:18:34.440157,1115994996230693426,1,1,0
2,6333597102633388268,2019-03-11 14:18:35.862360,7463856250762200923,1,1,0
3,6333597102633388268,2019-03-11 14:18:36.167163,7829815350267792040,1,1,0
4,6333597102633388268,2019-03-11 14:18:37.728590,1448534231953777480,1,1,0


### Hecho un vistazo a los valores de 'platform' 'ref_type_id' 'source_id'

In [14]:
auctions.describe()

,country,device_id,platform,ref_type_id,source_id
count,1.957132e+07,1.957132e+07,1.957132e+07,1.957132e+07,1.957132e+07
mean,6.333597e+18,4.599151e+18,1.205888e+00,2.235326e+00,6.857898e-01
std,3.016674e+09,2.667138e+18,4.043488e-01,2.426093e+00,1.416215e+00
min,6.333597e+18,1.138588e+14,1.000000e+00,1.000000e+00,0.000000e+00
25%,6.333597e+18,2.258716e+18,1.000000e+00,1.000000e+00,0.000000e+00
50%,6.333597e+18,4.616600e+18,1.000000e+00,1.000000e+00,0.000000e+00
75%,6.333597e+18,6.911427e+18,1.000000e+00,1.000000e+00,1.000000e+00
max,6.333597e+18,9.223339e+18,2.000000e+00,7.000000e+00,6.000000e+00


#### Los valores minimo y maximo estan entre 0 y 7

In [15]:
auctions['platform'].value_counts()

1    15541825
2     4029494
Name: platform, dtype: int64

In [16]:
auctions['ref_type_id'].value_counts()

1    15541825
7     4029494
Name: ref_type_id, dtype: int64

In [17]:
auctions['source_id'].value_counts()

0    13354597
1     4016739
5     1466494
2      582083
6      151406
Name: source_id, dtype: int64

#### Solo tienen valores pequeños, no hace falta int64, lo cambio a int32

In [18]:
auctions[['platform','ref_type_id','source_id']] = auctions[['platform','ref_type_id','source_id']].astype('int32')

In [19]:
auctions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19571319 entries, 0 to 19571318
Data columns (total 6 columns):
country        int64
date           datetime64[ns]
device_id      int64
platform       int32
ref_type_id    int32
source_id      int32
dtypes: datetime64[ns](1), int32(3), int64(2)
memory usage: 671.9 MB


### Se bajó considerablemente el peso del dataframe de más de un GB a 670 MB

### Finalmente: 
#### -Descartar la columna auction_type_id ya que no tiene datos relevantes
#### -Cambiar el date a datetime 
#### -Cambiar los tipos de 'source_id', 'platform', 'ref_type_id' a int32

# Analisis de clicks

In [20]:
clicks = pd.read_csv('clicks.csv.gzip', compression='gzip')

In [21]:
len(clicks)

26351

In [22]:
clicks.head()

,advertiser_id,action_id,source_id,created,country_code,latitude,longitude,wifi_connection,carrier_id,trans_id,os_minor,agent_device,os_major,specs_brand,brand,timeToClick,touchX,touchY,ref_type,ref_hash
0,2,NaN,4,2019-03-06T22:42:12.755Z,6333597102633388268,1.205689,1.070234,False,1.0,iGgClCM9exiHF4K31g94XmvHEBSLKIY,6.768137e+18,NaN,3.072849e+18,2733035977666442898,NaN,1.563,0.905,0.078,1891515180541284343,1904083516767779093
1,0,NaN,0,2019-03-08T10:24:30.641Z,6333597102633388268,1.218924,1.071209,False,4.0,MMHTOJ6qKAOeIH_Eywh1KIcCaxtO9oM,3.025219e+18,NaN,1.774085e+18,392184377613098015,NaN,NaN,NaN,NaN,1891515180541284343,3086509764961796666
2,0,NaN,0,2019-03-08T15:24:16.069Z,6333597102633388268,1.205689,1.070234,False,6.0,vIrEIdf9izUaWdAri6Ezk7T3nHFvNQU,5.975656e+18,NaN,3.072849e+18,392184377613098015,NaN,NaN,0.946,0.473,1891515180541284343,6958163894863846647
3,2,NaN,3,2019-03-06T03:08:51.543Z,6333597102633388268,1.205689,1.070234,False,45.0,YaKxxEAs2UmZhSpRfiCO9Zpa82B_AKM,6.768137e+18,NaN,3.072849e+18,2733035977666442898,NaN,19.013,0.035,0.431,1891515180541284343,4368617728156436525
4,2,NaN,3,2019-03-06T03:32:55.570Z,6333597102633388268,1.205689,1.070234,False,45.0,X5XTOcYQovkl6yadYdAD7xioVGU9jiY,6.768137e+18,NaN,3.072849e+18,2733035977666442898,NaN,28.110,0.054,0.423,1891515180541284343,4368617728156436525


In [23]:
clicks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26351 entries, 0 to 26350
Data columns (total 20 columns):
advertiser_id      26351 non-null int64
action_id          0 non-null float64
source_id          26351 non-null int64
created            26351 non-null object
country_code       26351 non-null int64
latitude           26351 non-null float64
longitude          26351 non-null float64
wifi_connection    26351 non-null bool
carrier_id         26340 non-null float64
trans_id           26351 non-null object
os_minor           26339 non-null float64
agent_device       3243 non-null float64
os_major           26339 non-null float64
specs_brand        26351 non-null int64
brand              6235 non-null float64
timeToClick        22977 non-null float64
touchX             23011 non-null float64
touchY             23011 non-null float64
ref_type           26351 non-null int64
ref_hash           26351 non-null int64
dtypes: bool(1), float64(11), int64(6), object(2)
memory usage: 3.8+ MB


#### Cambiamos el tipo de 'created' a datetime:

In [24]:
clicks['created'] = pd.to_datetime(clicks['created'])

### 'action_id' tiene todos los valores nulos, lo descartamos.

In [25]:
clicks = clicks.drop(columns='action_id')

#### Voy a echar un vistazo sobre 'brand' y 'agent_device' que son los siguientes con más valores nulos

In [26]:
clicks['brand'].describe()

count    6235.000000
mean        1.482277
std         1.583764
min         0.000000
25%         0.000000
50%         1.000000
75%         2.000000
max        13.000000
Name: brand, dtype: float64

In [27]:
clicks['brand'].value_counts()

2.0     2027
0.0     1917
1.0     1521
4.0      240
3.0      186
5.0      129
6.0      124
7.0       32
9.0       31
8.0       16
10.0       8
12.0       3
13.0       1
Name: brand, dtype: int64

In [28]:
clicks['agent_device'].describe()

count    3.243000e+03
mean     4.171028e+18
std      2.460496e+18
min      4.513618e+16
25%      1.975183e+18
50%      3.516812e+18
75%      6.589622e+18
max      9.221733e+18
Name: agent_device, dtype: float64

In [29]:
clicks['agent_device'].value_counts()

3.516812e+18    656
1.754528e+18    326
6.882415e+18    156
7.787531e+18    132
4.281729e+18    122
4.445252e+18     99
6.589622e+18     87
2.066431e+17     81
3.686067e+17     75
8.656854e+18     74
5.990117e+18     72
7.116343e+18     70
1.063783e+18     68
2.616228e+18     64
8.163783e+18     62
8.604122e+18     47
3.977966e+18     46
2.438316e+18     43
4.430579e+18     33
1.181318e+18     32
5.960896e+18     28
1.540907e+18     28
8.535198e+18     23
3.432684e+18     23
3.051485e+18     22
1.321681e+18     22
2.293453e+18     21
1.966828e+18     20
6.558466e+18     20
7.628197e+18     19
               ... 
8.178177e+18      1
8.285870e+18      1
8.400757e+18      1
7.052220e+18      1
6.445596e+18      1
3.103697e+17      1
1.251416e+17      1
4.928434e+18      1
1.231797e+18      1
3.262918e+18      1
4.186571e+18      1
3.339732e+17      1
6.195314e+18      1
7.614489e+18      1
3.285184e+18      1
2.091433e+17      1
8.369550e+18      1
1.557569e+18      1
4.497716e+18      1


#### Vista de valores de la columna booleana, 'wifi_connection'

In [30]:
clicks['wifi_connection'].describe()

count     26351
unique        1
top       False
freq      26351
Name: wifi_connection, dtype: object

#### Se ve que todos los valores son False.

### Finalmente:
##### - Se cambia el tipo de dato de 'created' a datetime.
##### - Todos los valores de 'wifi_connection' son False.
##### -Solo se descarta 'action_id' por ser todos valores nulos, luego el DataFrame no ocupa mucha memoria como para que los demas datos molesten.

# Analisis de events

In [32]:
events = pd.read_csv('events.csv.gzip', compression='gzip')

In [33]:
events.head()

,date,event_id,ref_type,ref_hash,application_id,attributed,device_countrycode,device_os_version,device_brand,device_model,...,trans_id,user_agent,event_uuid,carrier,kind,device_os,wifi,connection_type,ip_address,device_language
0,2019-03-05 00:09:36.966,0,1891515180541284343,2688759737656491380,38,False,6333597102633388268,5.908703e+17,NaN,5.990117e+18,...,NaN,NaN,a9c0b263-acb2-4577-92c5-cbde5d7a5db1,2.248157e+17,5.516623e+18,7.531669e+18,NaN,Cable/DSL,7858558567428669000,4.077062e+17
1,2019-03-05 00:09:38.920,1,1891515180541284343,2688759737656491380,38,False,6333597102633388268,5.908703e+17,NaN,5.990117e+18,...,NaN,NaN,1cd98205-0d97-4ec2-a019-667997dbfe7a,2.248157e+17,9.977660e+17,7.531669e+18,NaN,Cable/DSL,7858558567428669000,4.077062e+17
2,2019-03-05 00:09:26.195,0,1891515180541284343,2688759737656491380,38,False,6333597102633388268,5.908703e+17,NaN,5.990117e+18,...,NaN,NaN,f02e2924-21ae-492b-b625-9021ae0a4eca,2.248157e+17,5.516623e+18,7.531669e+18,NaN,Cable/DSL,7858558567428669000,4.077062e+17
3,2019-03-05 00:09:31.107,2,1891515180541284343,2688759737656491380,38,False,6333597102633388268,5.908703e+17,NaN,5.990117e+18,...,NaN,NaN,a813cf45-a36e-4668-85e2-5395f1564e98,2.248157e+17,8.561153e+18,7.531669e+18,NaN,Cable/DSL,6324037615828123965,4.077062e+17
4,2019-03-09 21:00:36.585,3,1891515180541284343,2635154697734164782,38,False,6333597102633388268,7.391844e+18,NaN,5.960896e+18,...,NaN,NaN,63a4f0aa-e147-469f-8c55-4ca4f8d0e310,2.248157e+17,8.731902e+17,7.531669e+18,NaN,Cable/DSL,2894495631302821483,3.301378e+18


In [34]:
len(events)

2494423

In [35]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494423 entries, 0 to 2494422
Data columns (total 22 columns):
date                  object
event_id              int64
ref_type              int64
ref_hash              int64
application_id        int64
attributed            bool
device_countrycode    int64
device_os_version     float64
device_brand          float64
device_model          float64
device_city           float64
session_user_agent    float64
trans_id              object
user_agent            float64
event_uuid            object
carrier               float64
kind                  float64
device_os             float64
wifi                  object
connection_type       object
ip_address            int64
device_language       float64
dtypes: bool(1), float64(10), int64(6), object(5)
memory usage: 402.0+ MB


#### Convertimos los date a datetime:

In [36]:
events['date'] = pd.to_datetime(events['date'])

In [37]:
events.count()

date                  2494423
event_id              2494423
ref_type              2494423
ref_hash              2494423
application_id        2494423
attributed            2494423
device_countrycode    2494423
device_os_version     1022066
device_brand          1164963
device_model          2406456
device_city            614698
session_user_agent    2482637
trans_id                   82
user_agent            1391527
event_uuid            2489324
carrier                616434
kind                  2489324
device_os              657667
wifi                  1378872
connection_type        612463
ip_address            2494423
device_language       2406604
dtype: int64

In [38]:
events.isnull().sum()

date                        0
event_id                    0
ref_type                    0
ref_hash                    0
application_id              0
attributed                  0
device_countrycode          0
device_os_version     1472357
device_brand          1329460
device_model            87967
device_city           1879725
session_user_agent      11786
trans_id              2494341
user_agent            1102896
event_uuid               5099
carrier               1877989
kind                     5099
device_os             1836756
wifi                  1115551
connection_type       1881960
ip_address                  0
device_language         87819
dtype: int64

### Se ve que la columna 'trans_id' tiene una gran cantidad de null, conviene descartarla.
#### -También se ve que las columnas 'device_os_version', 'device_brand', 'device_city', 'user_agent', 'carrier', 'wifi y 'connection_type' tiene más o menos la mitad de valores nulos

In [39]:
events = events.drop(columns= 'trans_id')

### Observamos 'attributed' la columna booleana

In [40]:
events['attributed'].describe()

count     2494423
unique          2
top         False
freq      2489324
Name: attributed, dtype: object

#### La columna tiene una gran cantidad de False, habrá que ver si conviene descartarla.

In [41]:
events['attributed'].value_counts()

False    2489324
True        5099
Name: attributed, dtype: int64

### Veremos los valores de las columnas 'event_uuid', 'wifi' y 'connection_type' que son del tipo object, para concluir si conviene cambiar el tipo de dato.

In [42]:
events['event_uuid'].describe()

count                                  2489324
unique                                 2489324
top       b0eacc4e-2f3c-47a1-b346-9d1adb07f761
freq                                         1
Name: event_uuid, dtype: object

In [43]:
events['event_uuid'].head()

0    a9c0b263-acb2-4577-92c5-cbde5d7a5db1
1    1cd98205-0d97-4ec2-a019-667997dbfe7a
2    f02e2924-21ae-492b-b625-9021ae0a4eca
3    a813cf45-a36e-4668-85e2-5395f1564e98
4    63a4f0aa-e147-469f-8c55-4ca4f8d0e310
Name: event_uuid, dtype: object

#### 'event_uuid', tiene todos sus valores únicos.

In [44]:
events['wifi'].describe()

count     1378872
unique          2
top          True
freq       930902
Name: wifi, dtype: object

In [45]:
events['wifi'].value_counts()

True     930902
False    447970
Name: wifi, dtype: int64

In [46]:
events['wifi'].head(10)

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
6    NaN
7    NaN
8    NaN
9    NaN
Name: wifi, dtype: object

#### A traves del head no podemos observar claramente los valores, los filtramos:

In [47]:
events['wifi'][(events['wifi'].isnull()==False)]

154         True
155         True
156        False
157         True
158         True
159         True
160         True
161         True
162         True
163         True
164         True
165         True
166         True
167         True
168         True
169         True
170         True
171         True
3304        True
3305        True
3306        True
3307        True
3308        True
3309        True
3310        True
3311        True
3312        True
31403       True
31404      False
31405      False
           ...  
2494393     True
2494394     True
2494395    False
2494396    False
2494397     True
2494398     True
2494399    False
2494400     True
2494401     True
2494402     True
2494403     True
2494404     True
2494405     True
2494406     True
2494407     True
2494408     True
2494409    False
2494410     True
2494411     True
2494412     True
2494413     True
2494414     True
2494415    False
2494416     True
2494417     True
2494418     True
2494419    False
2494420     Tr

#### Los valores no nulos, son booleanos, vamos a transformar el tipo de dato

In [48]:
events['wifi'] = events['wifi'].astype(bool)

#### 'connection_type'

In [49]:
events['connection_type'].describe()

count        612463
unique            3
top       Cable/DSL
freq         331948
Name: connection_type, dtype: object

In [50]:
events['connection_type'].head()

0    Cable/DSL
1    Cable/DSL
2    Cable/DSL
3    Cable/DSL
4    Cable/DSL
Name: connection_type, dtype: object

In [51]:
events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2494423 entries, 0 to 2494422
Data columns (total 21 columns):
date                  datetime64[ns]
event_id              int64
ref_type              int64
ref_hash              int64
application_id        int64
attributed            bool
device_countrycode    int64
device_os_version     float64
device_brand          float64
device_model          float64
device_city           float64
session_user_agent    float64
user_agent            float64
event_uuid            object
carrier               float64
kind                  float64
device_os             float64
wifi                  bool
connection_type       object
ip_address            int64
device_language       float64
dtypes: bool(2), datetime64[ns](1), float64(10), int64(6), object(2)
memory usage: 366.3+ MB


##### Es una String, se lo deja como esta

### Finalmente:
##### -Se ve que la columna 'trans_id' tiene una gran cantidad de null(solo 82 con valor), conviene descartarla.
##### -También se ve que las columnas 'device_os_version', 'device_brand', 'device_city', 'user_agent', 'carrier', 'wifi y 'connection_type' tiene más o menos la mitad de valores nulos. En principio supongo que siendo un DataFrame de eventos, dependiendo del evento tendra sentido tener un valor asignado o null.
##### -Se cambio el tipo de dato de 'wifi' a booleano.
##### - La columna attributed tiene gran cantidad de valores False, solo 5000 True.

# Análisis de Installs

In [52]:
installs= pd.read_csv('installs.csv.gzip', compression= 'gzip')

In [53]:
installs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3412 entries, 0 to 3411
Data columns (total 18 columns):
created               3412 non-null object
application_id        3412 non-null int64
ref_type              3412 non-null int64
ref_hash              3412 non-null int64
click_hash            0 non-null float64
attributed            3412 non-null bool
implicit              3412 non-null bool
device_countrycode    3412 non-null int64
device_brand          1047 non-null float64
device_model          3411 non-null float64
session_user_agent    3364 non-null object
user_agent            1729 non-null object
event_uuid            865 non-null object
kind                  865 non-null object
wifi                  1729 non-null object
trans_id              6 non-null object
ip_address            3412 non-null int64
device_language       3378 non-null float64
dtypes: bool(2), float64(4), int64(5), object(7)
memory usage: 433.2+ KB


In [54]:
len(installs)

3412

#### Por lo visto en el .info 'click_hash' tiene todos nulos y 'trans_id' solamente 6 no nulos.

In [55]:
installs.head()

,created,application_id,ref_type,ref_hash,click_hash,attributed,implicit,device_countrycode,device_brand,device_model,session_user_agent,user_agent,event_uuid,kind,wifi,trans_id,ip_address,device_language
0,2019-03-13 01:43:33.445,0,1891515180541284343,8464844987297247076,NaN,False,False,6333597102633388268,3.083059e+17,4.445252e+18,NaN,NaN,NaN,NaN,NaN,NaN,5529284367937637301,NaN
1,2019-03-13 04:58:35.078,0,1891515180541284343,3250564871270161533,NaN,False,False,6333597102633388268,3.083059e+17,4.445252e+18,NaN,NaN,NaN,NaN,NaN,NaN,6098022721363533304,NaN
2,2019-03-13 04:20:57.666,0,1891515180541284343,7953318831018100268,NaN,False,True,6333597102633388268,5.137992e+17,6.026543e+18,HasOffers Mobile AppTracking v1.0,NaN,0bfa9b4c-e07a-4522-b32d-138e3165ac98,Open,NaN,NaN,4636407223847323867,3.301378e+18
3,2019-03-13 04:20:57.698,0,1891515180541284343,7953318831018100268,NaN,False,False,6333597102633388268,5.137992e+17,6.026543e+18,NaN,NaN,NaN,NaN,NaN,NaN,4636407223847323867,NaN
4,2019-03-10 22:24:56.684,0,1891515180541284343,8355110941557237501,NaN,False,True,6333597102633388268,1.083369e+18,7.876612e+18,HasOffers Mobile AppTracking v1.0,NaN,8f235a8c-41bf-4399-b33d-c41577c949b0,Open,NaN,NaN,685400811232710215,3.301378e+18


In [56]:
installs = installs.drop(columns = 'click_hash')

#### Cambio el tipo de 'created' a datetime

In [57]:
installs['created'] = pd.to_datetime(installs['created'])

In [58]:
installs['wifi'].value_counts()

True     1377
False     352
Name: wifi, dtype: int64

#### Cambio el tipo de dato de 'wifi' a bool

In [59]:
installs['wifi']= installs['wifi'].astype(bool)

#### 'implict' y 'attributed' columnas booleanas

In [60]:
installs['implicit'].value_counts()

False    2547
True      865
Name: implicit, dtype: int64

In [61]:
installs['attributed'].value_counts()

False    3412
Name: attributed, dtype: int64

##### Todos los valores de la columna 'attributed' son False

#### Analisis de 'application_id'

In [62]:
installs['application_id'].describe()

count    3412.000000
mean       10.247655
std         6.016194
min         0.000000
25%         7.000000
50%         9.000000
75%        10.000000
max        36.000000
Name: application_id, dtype: float64

In [63]:
installs['application_id'].value_counts()

7     947
9     731
10    389
16    362
8     328
2     174
29     97
20     95
6      35
1      34
26     28
34     28
15     20
0      18
3      17
28     17
12     15
18     14
24     13
17     13
14     13
21      7
5       3
13      3
33      2
30      2
19      2
32      2
36      1
4       1
23      1
Name: application_id, dtype: int64

##### No se observa nada relevante en la primer vista

### Finalmente
##### - Por lo visto en el .info 'click_hash' tiene todos nulos y 'trans_id' solamente 6 no nulos.
##### - Cambio el tipo de 'created' a datetime
##### - Todos los valores de la columna 'attributed' son False

#### Este último DataFrame no ocupa casi memoria, por lo que no hay que preocuparse ṕor este aspecto